In [ ]:
%matplotlib inline

import iris
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from wildfires.data.datasets import data_map_plot, GSMaP_dry_day_period

In [ ]:
import warnings
warnings.filterwarnings('ignore', '.*Collapsing a non-contiguous coordinate.*')

In [ ]:
dry_day_period = GSMaP_dry_day_period()

In [ ]:
cube = dry_day_period.cubes[0]
cube

In [ ]:
fig = data_map_plot(cube.collapsed('time', iris.analysis.MEAN))

In [ ]:
fig = data_map_plot(iris.analysis.maths.log(cube).collapsed('time', iris.analysis.MEAN))

In [ ]:
for single_time_cube in cube.collapsed('time', iris.analysis.MEAN).slices_over('time'):
    print(single_time_cube.coord('time').cell(0).point)
    plt.figure()
    plt.hist(single_time_cube.data)